The following code is intended to answer an assignment in the CS152 (AI) class at Minerva Schools at KGI.
It is a simple implementation of A* search for the solution of the Sliding-Tiles puzzle. The implementation should work for any board of size n*n, although it will mostly generate solutions in reasonable time for n =< 3.
A* search is a complete (it always reaches an answer, if on exists) and optimal (it reaches the best answer possible) algorithm when the proper heuristics are used.
"Proper" here means the heurstic needs to be _admissible_, i.e. it must never overestimate the distance (or value) to the goal.
A* is also expected to reach each node optimally (i.e. it will find the best possible way to reach this node) the first time it encounters it, if the heuristic is _consistent_, or _monotonic_, i.e. if any node expanded from a previous node has an equal or lower goal-distance (as evaluated by the heuristic) than the previous one.

To make the implementation smoother, we implement a priority queue. It is a minor adaptation from: https://docs.python.org/2/library/heapq.html?fbclid=IwAR0KGcNTfXyYLj1ZAhIAq5Ca3ZZQ-hyHHWMd7az28JOld62HAHTOFoIf6mE
and it lets us update previously explored nodes in the frontier

In [ ]:
import itertools
from heapq import heappop, heappush
class PriorityQueue:
    def __init__(self):
        self.pq = []  # list of entries arranged in a heap
        self.entry_finder = {}  # mapping of tasks to entries
        self.REMOVED = '<removed-task>'  # placeholder for a removed task
        self.counter = itertools.count()  # unique sequence count

    def push(self, task, priority=0):
        'Add a new task or update the priority of an existing task'
        if task in self.entry_finder:
            self.remove(task)
        count = next(self.counter)
        entry = [priority, count, task]
        self.entry_finder[task] = entry
        heappush(self.pq, entry)

    def remove(self, task):
        'Mark an existing task as REMOVED.  Raise KeyError if not found.'
        entry = self.entry_finder.pop(task)
        entry[-1] = self.REMOVED

    def pop(self):
        'Remove and return the lowest priority task. Raise KeyError if empty.'
        while self.pq:
            priority, count, task = heappop(self.pq)
            if task is not self.REMOVED:
                del self.entry_finder[task]
                return task
        raise KeyError('pop from an empty priority queue')
    
    def __len__(self):
        return len(self.entry_finder)

Ignore the imports for now, they will be explained later.
We begin by defining the PuzzleNode class which will let us store the states explored.
A 2D list is used to represent the grid/state.
max_frontier_size is not necessary and is only used for the purpose of the assignment.
A parent node is required for tracing the path to the solution,
steps and goal_distance are needed for estimating the distance to the goal by A*.
str(PuzzleNode) is used for the purpose of the assignment, although it is also useful for properly storing nodes while we later search them with A*.

In [ ]:
from collections import deque


# answer for (1)
class PuzzleNode:
    def __init__(self, state, parent, max_frontier_size, steps, goal_distance):
        """
        :param state: 2D list representing the board
        :param parent: other puzzle node, None for starting_node
        :param steps: number of steps to reach this node
        :param max_frontier_size: the easiest way to map frontier
                sizes to nodes is to store them here.
                Gives the max at time of exploration
        :param goal_distance: a heuristic applied on current state
        """
        self.state, self.parent, self.max_frontier_size, self.steps, self.goal_distance = \
            state, parent, max_frontier_size, steps, goal_distance
        return
    
    def __str__(self):
        """
        :return: the board game in string format
                    Each row separated by a new line
        """
        ans = []
        for row in self.state:
            ans.append(','.join([str(x) for x in row]))
        return '\n'.join(ans)

The following are two possible heuristics for A*
Neither over-estimates the result, so they are both admissible.
Misplaced Tiles is _not consistent_ – a tile can be moved (adding a step) while not changing the number of misplaced tiles. Thus a move may increase the evaluation of A*.
Manhattan Distance _is_ consistent – a moved tile always reduces the distance by at least 1, and increases the steps by 1.

In [ ]:
from collections import deque

# answer to (3).(b)
def misplacedTiles(state):
    """
    counts the number of tiles in their place.
    the use of deque here brings us down from O(n^2) or even O(2n^2)
    to O(n)
    """
    n = len(state)  # since we know the state is valid
    lst = deque([x for x in range(n * n)])  # que with all expected elements
    count = 0
    # iterates over the grid and adds 1 for each displaced tile
    for row in state:
        for tile in row:
            if tile != lst.popleft():  # pops each tested tile from the que
                count += 1
    return count


# answer to (3).(b)
def manhattanDistance(state):
    """
    finds the distance of each tile from its intended location
    returns a sum of all these distances
    """
    ans = 0
    n = len(state)  # since we know the state is valid
    for i, row in enumerate(state):
        # enumerate provides us with an index as well as a value
        for j, tile in enumerate(row):
            if tile != i * n + j:
                """
                if a tile is misplaced, we need to -
                1. figure out where it should have been
                2. find out how far it is from where it is
                3. add this manhattan distance to ans

                tile//n provides a truncated result: intended row
                tile%n provides a modulo: intended column
                taking the absolute value of the difference
                from the intended and the current is the manhattan distance 
                """
                ans += abs(i - tile // n) + abs(j - tile % n)
    return ans

The validator function will be called by our puzzleSolver function to validate that the problem is defined according to the rules – the grid must be of size n*n, and it must contain all numbers between 0 and n*n-1 exactly once. We could allow decimal fractions, but for ease of use this is only allowe in cases with a decimal of 0, that is, when the number is an integer.
To expand on this validator, we could have it check if the puzzle is solveable, instead of running it to completion first.

In [ ]:
# answer to (3).(a)
def validator(n, grid):
    """
    validater that the grid, or board, is of n*n format.
    Expected complexity: O(n), checking each tile at most once.
    tiles can be of float-format, if the float is x.0
    we could also adjust this to allow for x.y, and truncate
    the float.
    :param n: expected number of rows and columns
    :param grid: the state of the board, in 2D list
    :return: True if valid, False otherwise
    """
    valid_tiles = set([x for x in range(n*n)])
    if len(grid) != n:  # check there are n rows, else false
        return False
    ans = set()
    for row in grid:
        if len(row) != n:  # check there are n columns, else false
            return False
        for tile in row:
            # check the tiles are in range or if exist twice
            if tile not in valid_tiles or tile in ans or tile < 0 or tile > n * n:
                return False
            ans.add(tile) # add to the observed tile set
    return True

The A* search uses two supporting functions: deep_clone, which copies game states, and explore_next which expands nodes to create new game states.
There is an incorrect assumption in the code below, that the heuristic _is consistent_. If this assumption is false, the code may be expanding states sub-optimally, since a previously explored state can be re-explored with a lower goal-distance value.
Solutions can be implemented: first we could validate that the heuristic is consistent (not recommeneded). Second, we could use memoization to store previously explored states and update them as needed.

In [ ]:
def deep_clone(state):
    """
    :param state: takes a game state
    :return: output the same game state
                without modifying the original
    """
    new_state = []
    for row in state:
        new_state.append(row[:])
    return new_state


def explore_next(parent, heuristic, explored_set):
    """
    checks all states that can be reached from the current state
    according to the rules that only the 0 can move on y/x axis
    this is equivalent to moving any tile adjacent
        to the empty tile.
    :param parent: puzzle node to be expanded,
                    used for generating a new puzzle node
    :param heuristic: a goal-distance heuristic
    :param explored_set: all previously observed states
    :return: returns all nodes that can be reached from the parent
                excluding any previously observed states.
    """
    new_nodes = []
    # a list of possible "moves" in a 2D array
    moves = [(1, 0), (0, 1), (-1, 0), (0, -1)]

    for i, row in enumerate(parent.state):
        for j, tile in enumerate(row):
            if tile == 0:
                for ni, nj in moves:
                    """
                    we know len(state) is n,
                    because we validated before starting.
                    avoids out of range by checking i and j after
                    each addition/subtraction.
                    the new state is then generated by switching
                    the 0 with another adjacent tile.
                    we then test if the new state is in the
                    previously explored set. We do this with a test
                    node before applying the heuristic, since
                    applying the heuristic to a previously explored
                    state is a redundant process.
                    """
                    if 0 <= i + ni < len(parent.state) and 0 <= j + nj < len(parent.state):
                        explored_state = deep_clone(parent.state)  # copy the array
                        explored_state[i][j] = explored_state[i + ni][j + nj]
                        explored_state[i + ni][j + nj] = 0
                        test_node = PuzzleNode(explored_state, parent, parent.max_frontier_size, parent.steps + 1, -1)
                        """
                        doing this to save heuristic runs, it's expensive, man.
                        max_frontier_size will update when explored
                        answer to (3).(c) – avoiding repeated states
                        for "It should also check if a better path to a
                        previously-visited state has been found at any
                        search step and modify the frontier" – A* with
                        consistent heuristics assumes that a node is
                        always reached optimally. So this is not needed.
                        """
                        test_state = str(test_node)

                        if test_state in explored_set:
                            if explored_set[test_state].steps > test_node.steps:
                                test_node.goal_distance = explored_set[test_state].goal_distance
                                explored_set[test_state] = test_node
                                new_nodes.append(test_node)
                        if test_state not in explored_set:
                            test_node.goal_distance = heuristic(explored_state)
                            new_nodes.append(test_node)
                break
    return new_nodes


# answer for (3)
def a_star(start_state, heuristic):
    """
    takes a starting state, lists all substates reachable from it,
    adds them to frontier. The next state with the lowest cost
    evaluated by adding the heuristic distance and the number of
    steps is then selected for exploration.
    States previously explored are not added to the frontier,
    since we operate with the assumption that they were previously
    reached optimally.

    The goal state is achieved when heuristic(state) == 0.
    Each expansion adds 1 to the steps because of the game rules.
    Heap-Queue is used because it lets us keep the lowest-cost node
    at the start of the data structure for expansion.
    To make this work, nodes are stored in a tuple with this value.

    Since heapq doesn't deal well with our node structure,
    I added a dictionary mapping each state to a node,
    this allows us to retrieve nodes at O(1) for other uses.
    It also serves as the 'list' of explored states.
    """
    node_dict = {}
    start_node = PuzzleNode(start_state, None, 1, 0, heuristic(start_state))
    node_dict[str(start_node)] = start_node
    frontier = PriorityQueue()
    frontier.push(str(start_node), start_node.steps + start_node.goal_distance)
    max_frontier = len(frontier)
    # we use the str() function to only keep the state

    solution = []

    while frontier:
        # keep going till all nodes are explored, or a goal state is reached
        cur = frontier.pop()
        node_dict[cur].max_frontier_size = max(len(frontier), max_frontier)
        cur = node_dict[cur]  # pull the node from the dict
        if cur.goal_distance == 0:
            """
            this is the only termination condition except for
            unreachable solution. It works even if the start node
            is an end state.
            It then adds all nodes to the list 'solution' in reverse
            so solution[-1] is the start node.
            """
            while cur is not None:
                solution.append(cur)
                cur = cur.parent
            break
        for next_node in explore_next(cur, heuristic, node_dict):
            """
            explores the next node in the frontier using
            explore_next. Adds each new node to the dict
            then adds to the frontier, which arranges itself as heap 
            """
            key = str(next_node)
            node_dict[key] = next_node
            frontier.push(key, cur.steps + start_node.goal_distance)
        max_frontier = max(len(frontier), max_frontier)
        # not expecting this to be different from len(frontier)
        # in most cases, but it could happen.
    return solution, max_frontier  # solution is empty if there isn't one



solvePuzzle uses all the previously defined elements.
The prnt variable is not required and is only used for the purpose of this assignment. It prints out all the steps to the solution evaluated by our A* search. It prints the number of steps left to reach the solution, and the largest number of unexplored nodes so far (the frontier).

In [ ]:
# answer for (2)
def solvePuzzle(n, state, heuristic, prnt=False):
    """
    n - the puzzle dimension (i.e. n x n board)
    state - the starting (scrambled) state of the puzzle,
    provided as a list of lists, with the blank space represented
    by the number 0. For example, for n=3, we could have.
    state = [[7 2 4],[5 0 6],[8 3 1]] as in the image shown previously.
    heuristic - a handle to a heuristic function (more details in the next step)
    prnt - a boolean value that indicates whether or not to print the solution

    :return:
    steps - the number of steps to optimally reach the goal state from the initial state
    frontierSize - the maximum (i.e. worst-case) size of the frontier during the search
    err - an error code (details given in the next step)
    """
    steps, frontierSize, err = 0, 0, 0

    if not validator(n, state):
        err = -1  # if the state is invalid, we stop early and return an error
        return steps, frontierSize, err

    solution, frontierSize = a_star(state, heuristic)

    if solution:
        # answer to (3).(d)
        if prnt:
            count = 1
            rng = len(solution)
            for pos in range(rng):
                node = solution[-pos]
                print(str(node))
                print(str(rng-count) + " steps to goal")
                print("max frontier size is: " + str(node.max_frontier_size))
                count += 1

        return solution[0].steps, frontierSize, err

    # there's a better way to run this validation
    return 0, 0, -2

It would be even wiser to actually build test cases for this process, and test a variaty of n's, different sitautions with non-integers and incorrect list sizes, as well as unsolveable states.
For now, here are a few test cases:

In [ ]:
# list of heuristics
heuristics = [misplacedTiles, manhattanDistance]
test_cases = [[2,2],
              [[0,0,0],[1,2,3,4]],
              [[5,7,7],[2,4,3],[8,1,0]],
              [[5,7,6],[2,4,3.2],[8,1,0]],
              [[5,7,6],[2,4,3.2],["8",1,0]],
              [[5,7,6],[2,4,3.2],[[8],1,0]],
              [[5,7,6],[2,4,3],[8,1,0]],
              [[7,0,8],[4,6,1],[5,3,2]],
              [[2,3,7],[1,8,0],[6,5,4]],
              [[0,1,2],[3,4,5],[6,7,8]]]

n = 3
prnt = False
for heuristic in heuristics:
    for test_case in test_cases:
        steps, frontierSize, err = solvePuzzle(n, test_case, heuristic, prnt)
        print(steps, frontierSize, err)

Results for both manhattan distance and misplaced tiles are identical.